In [90]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [91]:
import win32com.client as cli

In [2]:
xw.books

Books([<Book [cytiva.xlsm]>])

In [4]:
xw.Book('../print_form.xlsx')

<Book [print_form.xlsx]>

In [6]:
form = '{"meta": {"desc": "service_part_reqeust", "cols": {"fe_name": "fe_이름", "fe_initial": "trunkstock_id", "req_day": "요청일", "req_time": "요청시간", "address": "주소", "del_met": "배송방법", "is_return": "왕복배송여부", "recipient": "수령인", "is_urgent": "긴급여부", "parts": "요청파트", "del_instruction": "배송요청사항"}, "std_day": "2023-01-06"}, "data": ["홍길동", "KR_HGD", "2023-01-12", "18:00", "인천 연수구 송도동 218-3 삼성바이오로직스 5공장", "퀵", 0, "홍길동", 0, {"28999466": 2, "29159123": 1, "15921499": 1}, null]}'

In [7]:
import json

In [8]:
dict_form = json.loads(form)

In [9]:
dict_form.keys()
dict_form['meta']

{'desc': 'service_part_reqeust',
 'cols': {'fe_name': 'fe_이름',
  'fe_initial': 'trunkstock_id',
  'req_day': '요청일',
  'req_time': '요청시간',
  'address': '주소',
  'del_met': '배송방법',
  'is_return': '왕복배송여부',
  'recipient': '수령인',
  'is_urgent': '긴급여부',
  'parts': '요청파트',
  'del_instruction': '배송요청사항'},
 'std_day': '2023-01-06'}

In [10]:
dict_form['data']

['홍길동',
 'KR_HGD',
 '2023-01-12',
 '18:00',
 '인천 연수구 송도동 218-3 삼성바이오로직스 5공장',
 '퀵',
 0,
 '홍길동',
 0,
 {'28999466': 2, '29159123': 1, '15921499': 1},
 None]

- Index E2
- Printed Day	E3
- Recipient	B6
- Contact	E6
- Request Date	A10
- Address	D10
- Deilevery Instructions	A14
- parts_info	B21


In [18]:
wb_pf = xw.Book('print_form.xlsx')

In [14]:
wb_pf.sheets[0].range("A1").value

## 프린트하기
- 양식채우기

In [19]:
wb_cy
ws_svc = wb_pf.sheets['SVC']


In [205]:
from datetime import datetime

In [111]:
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [86]:
str(now.date())

'2023-01-13'

In [206]:
form_data = dict_form['data']
form_data

{'fe_name': '홍길동',
 'fe_initial': 'KR_HGD',
 'req_day': '2023-01-12',
 'req_time': '18:00',
 'address': '인천 연수구 송도동 218-3 삼성바이오로직스 5공장',
 'del_met': '퀵',
 'is_return': 0,
 'recipient': '홍길동',
 'is_urgent': 0,
 'parts': {'11000322': 2, '18117251': 1, '28404190': 1},
 'del_instruction': None,
 'contact': '010-1234-1234'}

In [215]:
list(form_data['parts'].values())

[2, 1, 1]

In [240]:

is_return = form_data['is_return']

if is_return == 0 :
    is_return = None:
else :
    is_return = '왕복'

0

In [ ]:
is_urgent = form_data['is_urgent']

if is_urgent == 0 :
    is_urgent = None:
else :
    is_urgent = '긴급'


### parts JSON
1. part_no
2. serial_no
3. qty
4. currnet_stock

In [260]:
cols = ['part_no', 'serial_no', 'qty', 'currnet_stock','transfer_to','BIN']
part_no = ['11000322','18117251','28404190']
serial_no = [None,None,None]
qty = [2,1,1]
currnet_stock = ['KR_SERV01','KR_SERV01','KR_SERV01']
transfer_to = ['KR_HGD','KR_HGD','KR_HGD']
BIN= [None,None,None]
df = pd.DataFrame([part_no,serial_no,qty,currnet_stock,transfer_to,BIN],index=cols).T
df

,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
0,11000322,None,2,KR_SERV01,KR_HGD,None
1,18117251,None,1,KR_SERV01,KR_HGD,None
2,28404190,None,1,KR_SERV01,KR_HGD,None


In [261]:
data=[]
for i in range(len(part_no)):
    tmp = zip(cols,[part_no[i],serial_no[i],qty[i],currnet_stock[i],transfer_to[i],BIN[i]])
    data.append(dict(tmp))
data

[{'part_no': '11000322',
  'serial_no': None,
  'qty': 2,
  'currnet_stock': 'KR_SERV01',
  'transfer_to': 'KR_HGD',
  'BIN': None},
 {'part_no': '18117251',
  'serial_no': None,
  'qty': 1,
  'currnet_stock': 'KR_SERV01',
  'transfer_to': 'KR_HGD',
  'BIN': None},
 {'part_no': '28404190',
  'serial_no': None,
  'qty': 1,
  'currnet_stock': 'KR_SERV01',
  'transfer_to': 'KR_HGD',
  'BIN': None}]

In [276]:
json.dumps(data,ensure_ascii=False,)

'[{"part_no": "11000322", "serial_no": null, "qty": 2, "currnet_stock": "KR_SERV01", "transfer_to": "KR_HGD", "BIN": null}, {"part_no": "18117251", "serial_no": null, "qty": 1, "currnet_stock": "KR_SERV01", "transfer_to": "KR_HGD", "BIN": null}, {"part_no": "28404190", "serial_no": null, "qty": 1, "currnet_stock": "KR_SERV01", "transfer_to": "KR_HGD", "BIN": null}]'

In [262]:
cols = ['part_no', 'serial_no', 'qty', 'currnet_stock','transfer_to','BIN']

res =  {
    'meta':{
        'desc':'요청파트정보',
        'cols':{
            'part_no':'파트번호',
            'serial_no':'시리얼넘버',
            'qty':'개수',
            'currnet_stock':'현재시스템로케이션위치',
            'transfer_to':'Transfer해야할 시스템로케이션',
            'BIN':'파트물리적위치'
        }
        
    },
    'parts':data
    
}

parts = json.dumps(res,ensure_ascii=False)
json_parts = json.loads(parts)

In [263]:
df_parts = pd.DataFrame(json_parts['parts'])
df_parts.index = df_parts.index + 1

In [264]:
df_parts
## Transfer_to, BIN 컬럼이 추가적으로 필요함


,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
1,11000322,None,2,KR_SERV01,KR_HGD,None
2,18117251,None,1,KR_SERV01,KR_HGD,None
3,28404190,None,1,KR_SERV01,KR_HGD,None


In [265]:
bin_loc = wb_cy.sheets['Cytiva Inventory BIN']
bin_loc

<Sheet [cytiva.xlsm]Cytiva Inventory BIN>

In [266]:
bin_last_row = bin_loc.range('A100000').end('up').row
bin_last_col = bin_loc.range('AAA'+str(bin_last_row)).end('left').column
bin_last_col

4

In [267]:
df_bin= pd.DataFrame(bin_loc.range((2,1),(bin_last_row,bin_last_col)).options(numbers=int).value)

In [268]:
dict_bin = dict(zip(df_bin[0].astype(str),df_bin[1]))

In [269]:
bin_list = []
for part_no in df_parts[df_parts.columns[0]]:
    bin_list.append(dict_bin[part_no])

In [270]:
bin_list

['2B1702', '2C1803', '2B1804']

In [271]:
df_parts['BIN'] = bin_list

In [272]:
df_parts

,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
1,11000322,None,2,KR_SERV01,KR_HGD,2B1702
2,18117251,None,1,KR_SERV01,KR_HGD,2C1803
3,28404190,None,1,KR_SERV01,KR_HGD,2B1804


In [259]:
# Index
# 요청출고의 인덱스번호로결정 1 로가정
ws_svc.range('E2').value = 1
# Printed Day E3
ws_svc.range('E3').value = now
# Delievery Type E5
is_return = form_data['is_return']
if is_return == 0 :
    is_return = None
else :
    is_return = '왕복'
ws_svc.range('E5').value = is_return
ws_svc.range('E3').value = now
# Recipient B6
ws_svc.range('B6').value = form_data['recipient']
# Contact E6
ws_svc.range('E6').value = form_data['contact']
# Request day A10
ws_svc.range('A10').value  = form_data['req_day']
# request hour A11
ws_svc.range('A11').value  = form_data['req_time']
# Address D10
ws_svc.range('D10').value = form_data['address']
# Deilevery Instructions A14
ws_svc.range('A14').value = form_data['del_instruction']
# parts_info A20
ws_svc.range('A20').value = df_parts

# Depart From B41
ws_svc.range('B41').value = '서울시 강서구 하늘길 247 3층 C구역'
# TEL B42
ws_svc.range('B42').value = '02-2660-3767'
# Phone B43
ws_svc.range('B43').value = '담당자 폰번호'
# URGENT E41
is_urgent = form_data['is_urgent']
if is_urgent == 0 :
    is_urgent = None       
else :
    is_urgent = '긴급' 
ws_svc.range('E41').value = is_urgent

In [289]:
import win32com.client as cli

In [300]:
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
msg = inbox.Items #메세지 정보
msg_counter = msg.count # 총갯수

part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

SVC_20230114_CCH_noturgent


In [299]:
body_exam = part_request[0].Body
body = body_exam[:body_exam.rfind('}')+1]
body
json_body = json.loads(body)

json_body.keys()

form_data= json_body['data']
df_parts = pd.DataFrame(form_data['parts'])
df_parts

,part_no,serial_no,qty,currnet_stock,transfer_to,BIN
0,11000322,None,2,KR_SERV01,KR_HGD,None
1,18117251,None,1,KR_SERV01,KR_HGD,None
2,28404190,None,1,KR_SERV01,KR_CCH,None


In [302]:
dict_data

'인천 연수구 송도동 218-3 삼성바이오로직스 5공장'

In [310]:
df_parts = pd.DataFrame(dict_data['parts'])


In [333]:
part_request[0].Subject+'_prfm'

'SVC_20230114_CCH_noturgent_prfm'

In [334]:
def print_form(subject=None):
    """
    """
       
    wb_pf = xw.Book('print_form.xlsx')
    ws_svc = wb_pf.sheets['SVC']
    
    outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
    inbox = outlook.GetDefaultFolder(6) # 받은편지함
    
    
    part_request = []
    for ms in inbox.Items:
        if subject in ms.Subject:
            part_request.append(ms)
            
    
    if 'SVC' in part_request[0].Subject:
    

        body_exam = part_request[0].Body
        body = body_exam[:body_exam.rfind('}')+1]
        body
        json_body = json.loads(body)

        json_body.keys()

        form_data= json_body['data']
        df_parts = pd.DataFrame(form_data['parts'])
        df_parts
        # Index

        ws_svc.range('B21:H40').clear_contents()


        # 요청출고의 인덱스번호로결정 1 로가정
        ws_svc.range('E2').value = 1
        # Printed Day E3
        ws_svc.range('E3').value = now
        # Delievery Type E5
        is_return = form_data['is_return']
        if is_return == 0 :
            is_return = None
        else :
            is_return = '왕복'
        ws_svc.range('E5').value = is_return
        ws_svc.range('E3').value = now
        # Recipient B6
        ws_svc.range('B6').value = form_data['recipient']
        # Contact E6
        ws_svc.range('E6').value = form_data['contact']
        # Request day A10
        ws_svc.range('A10').value  = form_data['req_day']
        # request hour A11
        ws_svc.range('A11').value  = form_data['req_time']
        # Address D10
        ws_svc.range('D10').value = form_data['address']
        # Deilevery Instructions A14
        ws_svc.range('A14').value = form_data['del_instruction']
        # parts_info A20
        ws_svc.range('A20').value = df_parts
        # Depart From B41
        ws_svc.range('B41').value = '서울시 강서구 하늘길 247 3층 C구역'
        # TEL B42
        ws_svc.range('B42').value = '02-2660-3767'
        # Phone B43
        ws_svc.range('B43').value = '담당자 폰번호'
        # URGENT E41
        is_urgent = form_data['is_urgent']
        if is_urgent == 0 :
            is_urgent = None       
        else :
            is_urgent = '긴급' 
        ws_svc.range('E41').value = is_urgent




In [322]:
print_form()

## 프린팅하기

In [25]:
wb_pf.sheets['SVC'].range("A1:H43").api.PrintOut()

True

In [27]:
wb_pf.sheets['SVC'].range("A1:H43").api.PrintPreview()

True

# print_form.xlsx 안켜져있어도 프린트 기능 하기

In [26]:
import os
import sys

In [85]:
pf_dir = "C:\\Users\\lms46\\Desktop\\fulfill\\print_form.xlsx"


In [80]:
form_dir = os.path.join(xl_dir[0]+"\\",xl_dir[1],xl_dir[2],xl_dir[3],xl_dir[4])

In [86]:
xw.Book(xl_dir)

<Book [print_form.xlsx]>

In [87]:
os.getcwd()

'C:\\Users\\lms46\\Desktop\\fulfill'

In [159]:
import time

# 메일 연동 방법
1. 시스템으로 발송된 메일은 자동으로 DB에 내용 저장 mail_detail

In [187]:
# 다른 폴더 메일
        
request_folder = outlook.GetDefaultFolder(6).Parent.Folders('1_Requests')
process_folder = outlook.GetDefaultFolder(6).Parent.Folders('2_Processing')
shipped_folder = outlook.GetDefaultFolder(6).Parent.Folders('3_ShipConfirmed')

In [176]:
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함


part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903


In [188]:
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)
for ms in part_request:
    if 'SVC' in ms.Subject:
        print(ms)
        ms.Move(request_folder)

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903
SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903


In [189]:
# 2번폴더로 이동
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
request_folder = outlook.GetDefaultFolder(6).Parent.Folders('1_Requests') # 받은편지함
part_request = []
for ms in request_folder.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)
        
for ms in part_request:
    if 'SVC' in ms.Subject:
        print(ms)
        ms.Move(process_folder)

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903
SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903


In [190]:
# 3번폴더로 이동
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
process_folder = outlook.GetDefaultFolder(6).Parent.Folders('2_Processing') # 받은편지함
part_request = []
for ms in process_folder.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)
        
for ms in part_request:
    if 'SVC' in ms.Subject:
        print(ms)
        ms.Move(shipped_folder)

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903
SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903


In [193]:
# 폴더이동 모듈
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6) # 받은편지함
request_folder = outlook.GetDefaultFolder(6).Parent.Folders('1_Requests') # 1번폴더
process_folder = outlook.GetDefaultFolder(6).Parent.Folders('2_Processing') # 2번폴더
shipped_folder = outlook.GetDefaultFolder(6).Parent.Folders('3_ShipConfirmed') # 3번폴더

def move_mail(from_folder,to_folder,req_type="SVC"):
    part_request = []
    for ms in from_folder.Items:
        if req_type in ms.Subject:
            part_request.append(ms)
    
    # from_folder내용이 없을 경우
    if len(part_request) == 0:
        return None
    else :
        for ms in part_request:
            if req_type in ms.Subject:
                ms.Move(to_folder)



In [197]:
move_mail(inbox,process_folder)

In [200]:
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함


part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903
SVC_LWW_23012101


# DB 테이블

## MAIL_STATUS
- **MS_INDEX(PK)**
- ML_SUB(FK)
- ML_STATUS  --> ['REQUESTED','PROCESSING', 'SHIPPED']
- ML_BIN --> ['1_Requests', '2_Processing', '3_ShipConfirmed']
- STD_DAY(분까지)

## MAIL_DETAIL
- **ML_SUB(PK)**
- ML_TYPE_NM(FK)
- STD_DAY(분까지)
- ML_BODY

## MAIL_TYPE
- **ML_TYPE_NM(PK)**
- TYPE_NAME
- 0:'SVC', 1:'SO', 2:'DEMO', 99:'else'

In [243]:
from datetime import datetime as dt

In [220]:
outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함


part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903


In [246]:
### MAIL_DETAIL

# ML_SUB(PK)
part_request[0].Subject

# ML_TYPE_NM
0

# STD_DAY
str(part_request[0].CreationTime).split('.')[0]

# ML_BODY
part_request[0].Body.split("\r\n")[0]



#############################################################################
#############################################################################

### MAIL_STATUS


# MS_INDEX(PK) 
# 자동 증분

# ML_SUB(FK)
part_request[0].Subject

# ML_STATUS
ml_status[0]

# ML_BIN
ml_folders[0]

# STD_DAY(분까지)
str(dt.now()).split('.')[0]

'2023-01-21 16:22:42'

In [347]:
def get_insert_values(argu):
    list_len = len(argu)

    value_list = []
    for i in range(1,list_len+1):
        value_list.append(":"+str(i))
    str_val = ','.join(value_list)
    
    return str_val


In [360]:

outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
request_folder = outlook.GetDefaultFolder(6).Parent.Folders('1_Requests') # 1번폴더
process_folder = outlook.GetDefaultFolder(6).Parent.Folders('2_Processing') # 2번폴더
shipped_folder = outlook.GetDefaultFolder(6).Parent.Folders('3_ShipConfirmed') # 3번폴더

# 폴더리스트
folder_list = [inbox,request_folder,process_folder,shipped_folder]

# 상태리스트
ml_status= ['REQUESTED','PROCESSING', 'SHIPPED']
ml_folders = ['1_Requests', '2_Processing', '3_ShipConfirmed']


# 테이블리스트 
tb_list = ['MAIL_DETAIL' ,'MAIL_STATUS']



part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

    df_list_md=[]
    df_list_ms=[]
    
for ms in part_request:
    query_list=[]

    tmp_md = []
    tmp_ms = []
    ### MAIL_DETAIL
    # ML_SUB(PK)
    tmp_md.append(ms.Subject)
    # ML_TYPE_NM
    tmp_md.append(0)
    # STD_DAY
    tmp_md.append(str(ms.CreationTime).split('.')[0])
    # ML_BODY
    tmp_md.append(ms.Body.split("\r\n")[0])
    #############################################################################
    ### MAIL_STATUS
    # MS_INDEX(PK) 
    # 자동 증분
#     tmp_ms.append(None)
    # ML_SUB(FK)
    tmp_ms.append(ms.Subject)
    # ML_STATUS
    tmp_ms.append(ml_status[0])
    # ML_BIN
    tmp_ms.append(request_folder.Name)
    # STD_DAY(분까지)
    tmp_ms.append(str(dt.now()).split('.')[0])
    query_list = [
    f'INSERT INTO MAIL_DETAIL ("ML_SUB", "ML_TYPE_NM", "STD_DAY", "ML_BODY") VALUES({get_insert_values(tmp_md)})',
    f'INSERT INTO MAIL_STATUS ("ML_SUB", "ML_STATUS", "UP_TIME", "ML_BIN") VALUES({get_insert_values(tmp_ms)})'        
    ]
    
    df_list_md.append(tmp_md)
    df_list_ms.append(tmp_ms)

    
df_md = pd.DataFrame(df_list_md).values.tolist()
df_tuple_md =list(pd.Series(df_list_md).map(tuple))
cursor.executemany(query_list[0],df_tuple_md,batcherrors= True)
cursor.execute("commit")

df_ms = pd.DataFrame(df_list_ms).values.tolist()
df_tuple_ms =list(pd.Series(df_list_ms).map(tuple))
cursor.executemany(query_list[0],df_tuple_ms,batcherrors= True)
cursor.execute("commit")

# #     print(query_list)
# #     print(tmp)
#     for idx,query in enumerate(query_list):
#         print(tuple(tmp[idx]))
#         cursor.executemany(query ,tuple(tmp[idx]))
#     cursor.execute("commit")

SVC_CCH_23011901
SVC_LMR_23011902
SVC_LWW_23011903


In [349]:
df_list = df_md.values.tolist()

In [350]:
df_tuple_md =list(pd.Series(df_list).map(tuple))
df_tuple
query_list

['INSERT INTO MAIL_DETAIL ("ML_SUB", "ML_TYPE_NM", "STD_DAY", "ML_BODY") VALUES(:1,:2,:3,:4)',
 'INSERT INTO MAIL_STATUS ("MS_INDEX", "ML_SUB", "ML_STATUS", "UP_TIME", "ML_BIN") VALUES(:1,:2,:3,:4,:5)']

In [351]:
cursor.executemany(query_list[0],df_tuple_md,batcherrors= True)

In [357]:
cursor.execute("commit")

In [356]:
df_tuple_ms

[(None, 'SVC_CCH_23011901', 'REQUESTED', '1_Requests', '2023-01-21 18:38:09'),
 (None, 'SVC_LMR_23011902', 'REQUESTED', '1_Requests', '2023-01-21 18:38:09'),
 (None, 'SVC_LWW_23011903', 'REQUESTED', '1_Requests', '2023-01-21 18:38:09')]

In [361]:
## ms 넣기
df_list = df_ms.values.tolist()
df_tuple_ms =list(pd.Series(df_list).map(tuple))

cursor.executemany(query_list[1],df_tuple_ms,batcherrors= True)
cursor.execute("commit")

In [302]:
tuple([1,2,3,4])

(1, 2, 3, 4)

In [362]:
# folder_list
move_mail(folder_list[0],folder_list[1])

In [ ]:
def move_mail(from_folder,to_folder,req_type="SVC"):
    part_request = []
    for ms in from_folder.Items:
        if req_type in ms.Subject:
            part_request.append(ms)
    
    # from_folder내용이 없을 경우
    if len(part_request) == 0:
        return None
    else :
        for ms in part_request:
            if req_type in ms.Subject:
                ms.Move(to_folder)

In [363]:
## insert용 values만들기 매서드
def get_insert_values(argu):
    list_len = len(argu)

    value_list = []
    for i in range(1,list_len+1):
        value_list.append(":"+str(i))
    str_val = ','.join(value_list)
    
    return str_val

In [ ]:
# 메일 excel 출력 함수
# "SVC" 한정

def 

outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
request_folder = outlook.GetDefaultFolder(6).Parent.Folders('1_Requests') # 1번폴더
process_folder = outlook.GetDefaultFolder(6).Parent.Folders('2_Processing') # 2번폴더
shipped_folder = outlook.GetDefaultFolder(6).Parent.Folders('3_ShipConfirmed') # 3번폴더

# 폴더리스트
folder_list = [inbox,request_folder,process_folder,shipped_folder]

# 상태리스트
ml_status= ['REQUESTED','PROCESSING', 'SHIPPED']
ml_folders = ['1_Requests', '2_Processing', '3_ShipConfirmed']


# 테이블리스트 
tb_list = ['MAIL_DETAIL' ,'MAIL_STATUS']



part_request = []
for ms in inbox.Items:
    if 'SVC' in ms.Subject:
        print(ms.Subject)
        part_request.append(ms)

    df_list_md=[]
    df_list_ms=[]
for ms in part_request:
    query_list=[]

    tmp_md = []
    tmp_ms = []
    ### MAIL_DETAIL
    # ML_SUB(PK)
    tmp_md.append(ms.Subject)
    # ML_TYPE_NM
    tmp_md.append(0)
    # STD_DAY
    tmp_md.append(str(ms.CreationTime).split('.')[0])
    # ML_BODY
    tmp_md.append(ms.Body.split("\r\n")[0])
    #############################################################################
    ### MAIL_STATUS
    # MS_INDEX(PK) 
    # 자동 증분
#     tmp_ms.append(None)
    # ML_SUB(FK)
    tmp_ms.append(ms.Subject)
    # ML_STATUS
    tmp_ms.append(ml_status[0])
    # ML_BIN
    tmp_ms.append(request_folder.Name)
    # STD_DAY(분까지)
    tmp_ms.append(str(dt.now()).split('.')[0])
    query_list = [
    f'INSERT INTO MAIL_DETAIL ("ML_SUB", "ML_TYPE_NM", "STD_DAY", "ML_BODY") VALUES({get_insert_values(tmp_md)})',
    f'INSERT INTO MAIL_STATUS ("ML_SUB", "ML_STATUS", "UP_TIME", "ML_BIN") VALUES({get_insert_values(tmp_ms)})'        
    ]
    
    df_list_md.append(tmp_md)
    df_list_ms.append(tmp_ms)

df_md = pd.DataFrame(df_list_md).values.tolist()
df_tuple_md =list(pd.Series(df_list_md).map(tuple))
cursor.executemany(query_list[0],df_tuple_md,batcherrors= True)
cursor.execute("commit")

df_ms = pd.DataFrame(df_list_ms).values.tolist()
df_tuple_ms =list(pd.Series(df_list_ms).map(tuple))
cursor.executemany(query_list[0],df_tuple_ms,batcherrors= True)
cursor.execute("commit")


move_mail(folder_list[0],folder_list[1])

# DB연동

In [247]:
import cx_Oracle
import os
import pandas as pd
#!pip insall cx_Oracle
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [325]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [319]:
def get_df_from_db(table_name='all_tables'):
    """
    table_name 에는 해당하는 table_name만 넣으면 자동으로 pandas 데이터 프레임으로 변환해준다.
    """
    rows_or = []
    try:
        for row_data in cursor.execute("select * from " + table_name):
            rows_or.append(list(row_data))
        colum_num = len(list(row_data))
    except: 
        print("Error -> 테이블명이 정확하지 않습니다.")
        raise
    num = int(colum_num)
    col_name = []
    for i in range(num):
        col_name.append('column_'+str(i))

    df = pd.DataFrame(columns=col_name)
    for idx,row in enumerate(rows_or):
        df.loc[idx] = row
    if table_name == 'all_tables' :
        df = df[df['column_0'] == 'DW_FULFILL'].rename(columns={'column_1':'조회가능한_테이블명'})[['조회가능한_테이블명']]. \
            reset_index().drop(columns='index')
        return df
    else :    
        return df

In [364]:
try:
    LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
    cx_Oracle.init_oracle_client(lib_dir=LOCATION)

    connection = cx_Oracle.connect(
        user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

    )
    cursor = connection.cursor()

except:
    connection = cx_Oracle.connect(
        user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

    )
    cursor = connection.cursor()
    
get_df_from_db()

,조회가능한_테이블명
0,SHIPMENT_INFORMATION
1,MAIL_STATUS
2,MAIL_DETAIL
3,MAIL_TYPE


In [ ]:
("ML_SUB", "ML_TYPE_NM", "STD_DAY", "ML_BODY")

In [365]:
cx_Oracle.init_oracle_client

<module 'cx_Oracle' from 'C:\\Users\\lms46\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\cx_Oracle.cp310-win_amd64.pyd'>

In [370]:
import sys

print(sys.version)

3.10.9 (tags/v3.10.9:1dd9be6, Dec  6 2022, 20:01:21) [MSC v.1934 64 bit (AMD64)]
